In [175]:
import pandas as pd
import numpy as np
import sympy as sp
from sympy import *
from sympy.abc import a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, alpha, beta, gamma, delta, epsilon, eta, theta, iota, kappa, mu, nu, xi, omicron, pi, rho, sigma, tau, upsilon, phi, chi, psi, omega
import matplotlib.pyplot as plt
import random
from IPython.display import Markdown

In [237]:
class EquationGenerator:
    def __init__(self):
        self.latin = symbols('a b c d e f g h i j k l m n o p q r s t u v w x y z')
        self.greek = symbols('alpha beta gamma delta epsilon zeta eta theta iota kappa mu nu xi omicron pi rho sigma tau upsilon phi chi psi omega')
        self.vars = self.latin + self.greek 
        self.operators = ('+', '-', '*', '/', '**')
        self.functions = (sin, cos, tan, exp, log, sqrt)
        self.nums = tuple(range(1, 10))

    def generate_expression(self):
        # complexity1 = random.randint(1, 3)
        self.vars += ("λ",)
        complexity1 = 1
        expression = random.choice(self.functions)(random.choice(self.vars))  # Start with a function of a variable
        for _ in range(complexity1 ** 2):
            operator = random.choice(self.operators)
            complexity2 = random.randint(1, 3)
            if complexity2 == 1:
                term = random.choice((random.choice(self.vars), random.choice(self.nums)))
            elif complexity2 == 2:
                term = random.choice(self.functions)(random.choice(self.vars))
            elif complexity2 == 3:
                func = random.choice(self.functions)
                inner1 = random.choice(self.vars)
                inner2 = random.choice((random.choice(self.nums), random.choice(self.vars)))
                operator = random.choice(self.operators)
                term = f"{func.__name__}({inner1} {operator} {inner2})"    
            expression = f"({expression}) {operator} ({term})" 
        print(expression)
        return sympify(expression)
    
    def combined_vars(self):
        part1 = random.choice(self.vars)
        part2 = random.choice(self.vars + self.nums)
        var = f"{part1}_{part2}"
        return symbols(var)
    
    def generate_equation(self):
        lhs = self.generate_expression()
        rhs = self.generate_expression()
        return Eq(lhs, rhs)
    
    def to_mathml(self, expr):
        mml = sp.printing.mathml(expr, printer='presentation')
        # mml = mml.replace("<mo>&InvisibleTimes;</mo>", "")
        # mml = mml.replace(">", ">\n")
        return mml
    



        
    
    



def main():
    help(sp.printing.mathml)
    eg = EquationGenerator()
    # eq = eg.generate_equation()
    eq = Eq(2*x*y + 3, 4*x + 5)
    latex_eq = latex(eq)
    mathml = eg.to_mathml(eq)
    print(mathml)
    display(Markdown(f"$$ {latex_eq} $$"))

if __name__ == "__main__":
    main()



Help on _PrintFunction in module sympy.printing.mathml:

mathml = <sympy.printing.printer._PrintFunction object>
    Returns the MathML representation of expr. If printer is presentation
    then prints Presentation MathML else prints content MathML.

<mrow><mrow><mrow><mn>2</mn><mo>&InvisibleTimes;</mo><mi>x</mi><mo>&InvisibleTimes;</mo><mi>y</mi></mrow><mo>+</mo><mn>3</mn></mrow><mo>=</mo><mrow><mrow><mn>4</mn><mo>&InvisibleTimes;</mo><mi>x</mi></mrow><mo>+</mo><mn>5</mn></mrow></mrow>


$$ 2 x y + 3 = 4 x + 5 $$

$$
\phi + \tan{\left(m \right)} = \cos^{5}{\left(g \right)}
$$